# Data Splitting and Saving

In [14]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split

In [15]:
# Path to your original dataset
data_dir = r'C:\Users\sandeep\OneDrive\Desktop\Navya_files\Mini Project\animals_dataset\animals\animals'

In [16]:
# Create directories for training, validation, and testing data
os.makedirs('train', exist_ok=True)
os.makedirs('validation', exist_ok=True)
os.makedirs('test', exist_ok=True)

In [17]:
# List all class directories
classes = os.listdir(data_dir)

In [19]:
# Define the ratio for splitting the data
train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

In [21]:
# Iterate over the subdirectories (classes) in your dataset
for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)
    image_files = os.listdir(class_dir)
    random.shuffle(image_files)  # Shuffle the images randomly
    # Split the images based on the defined ratios
    num_images = len(image_files)
    num_train = int(train_ratio * num_images)
    num_validation = int(validation_ratio * num_images)

    train_images = image_files[:num_train]
    validation_images = image_files[num_train:num_train + num_validation]
    test_images = image_files[num_train + num_validation:]
    
    
    # Split the images based on the defined ratios
    num_images = len(image_files)
    num_train = int(train_ratio * num_images)
    num_validation = int(validation_ratio * num_images)

    train_images = image_files[:num_train]
    validation_images = image_files[num_train:num_train + num_validation]
    test_images = image_files[num_train + num_validation:]

    # Create directories for each class in the train, validation, and test sets
    os.makedirs(os.path.join('train', class_name), exist_ok=True)
    os.makedirs(os.path.join('validation', class_name), exist_ok=True)
    os.makedirs(os.path.join('test', class_name), exist_ok=True)

    # Copy images to the respective directories
    for image in train_images:
        src = os.path.join(class_dir, image)
        dest = os.path.join('train', class_name, image)
        shutil.copy(src, dest)

    for image in validation_images:
        src = os.path.join(class_dir, image)
        dest = os.path.join('validation', class_name, image)
        shutil.copy(src, dest)

    for image in test_images:
        src = os.path.join(class_dir, image)
        dest = os.path.join('test', class_name, image)
        shutil.copy(src, dest)

# Loading a Deep Learning Model

In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

In [26]:
num_classes=len(data_dir)

In [27]:
# Define the data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(224, 224),  # Adjust the target size as needed
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    'validation',
    target_size=(224, 224),  # Adjust the target size as needed
    batch_size=32,
    class_mode='categorical'
)

Found 3759 images belonging to 90 classes.
Found 805 images belonging to 90 classes.


In [28]:
# Load a pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [29]:
# Add custom layers for your classification task
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [30]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10
118/118 [==============================] - 622s 5s/step - loss: 2.9379 - accuracy: 0.4195 - val_loss: 1.5316 - val_accuracy: 0.6758
Epoch 2/10
118/118 [==============================] - 526s 4s/step - loss: 0.6230 - accuracy: 0.8731 - val_loss: 1.0557 - val_accuracy: 0.7503
Epoch 3/10
118/118 [==============================] - 478s 4s/step - loss: 0.2070 - accuracy: 0.9691 - val_loss: 0.8060 - val_accuracy: 0.8112
Epoch 4/10
118/118 [==============================] - 878s 7s/step - loss: 0.0899 - accuracy: 0.9944 - val_loss: 0.6818 - val_accuracy: 0.8323
Epoch 5/10
118/118 [==============================] - 1075s 9s/step - loss: 0.0430 - accuracy: 0.9979 - val_loss: 0.6450 - val_accuracy: 0.8323
Epoch 6/10
118/118 [==============================] - 1038s 9s/step - loss: 0.0315 - accuracy: 0.9979 - val_loss: 0.5807 - val_accuracy: 0.8435
Epoch 7/10
118/118 [==============================] - 584s 5s/step - loss: 0.0202 - accuracy: 0.9997 - val_loss: 0.5425 - val_accuracy: 0.85

In [42]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
import numpy as np

# Load a pre-trained MobileNetV2 model (you can replace this with your custom model)
model = MobileNetV2(weights='imagenet')  # You can replace 'imagenet' with your custom model file if applicable.

# Define a function to preprocess an image for model input
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))  # Adjust target_size to match your model's requirements.
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

# Path to the image you want to classify
image_path =r"C:\Users\sandeep\OneDrive\Desktop\Navya_files\Mini Project\validation\cat\5e07a0666d.jpg"  # Replace with the actual path to your image.

# Preprocess the image
preprocessed_image = preprocess_image(image_path)

# Make predictions on the preprocessed image
predictions = model.predict(preprocessed_image)

# Decode the predictions to get class labels and probabilities
decoded_predictions = decode_predictions(predictions, top=1)  # You can adjust 'top' to get more or fewer predictions.

# Get the top predicted class label and its probability
predicted_class_label = decoded_predictions[0][0][1]
confidence = decoded_predictions[0][0][2]

print(f"Predicted class label: {predicted_class_label}")
print(f"Confidence: {confidence:.2f}")

1/1 [==============================] - 3s 3s/step
Predicted class label: Egyptian_cat
Confidence: 0.36


In [43]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import save_model

# Load the pre-trained MobileNetV2 model
model = MobileNetV2(weights='imagenet')

# Save the model to an HDF5 file
model.save('mobilenet_v2.h5')

print("Model saved to 'mobilenet_v2.h5'")

C:\Users\sandeep\anaconda3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to 'mobilenet_v2.h5'
